In [1]:
#rm(list = ls())
set.seed(100)
library(Rtsne)
library(RColorBrewer)
library(pheatmap)
library(dplyr)
library(ggplot2)
library(cowplot)
library(Seurat)
library(Matrix)



Attaching package: 'dplyr'

The following objects are masked from 'package:stats':

    filter, lag

The following objects are masked from 'package:base':

    intersect, setdiff, setequal, union


Attaching package: 'cowplot'

The following object is masked from 'package:ggplot2':

    ggsave



In [13]:
name <- "CZI_Fourth_Run_RNA_ADT_UMAP_paired_res_1.2_with_HTO_doubs"
# change directory to where your matrices will be stored
setwd("/projects/ucar-lab/danaco/Ground")
# read in genes
genes <- read.delim("genes.tsv",
                    header = F, stringsAsFactors = F, check.names = F)

# rna counts matrix
all_rna <- readRDS("CZI_RNA_Fourth_run_raw_matrices.sparse.Rds")
# remove genes with zeros across all cells (non-informative)
all_rna <- all_rna[rowSums(all_rna) > 0, ]


# change rownames of genes from ENSEMBL id to symbols
g_ids <- NULL
for (g in 1:nrow(all_rna)) {
  idx <- which(genes$V1 == rownames(all_rna)[g])
  g_ids <- c(g_ids, genes$V2[idx[1]])
}
rownames(all_rna) <- g_ids

# remove duplicate gene symbols from rna matrix
dup_id <- which(duplicated(rownames(all_rna)))
all_rna <- all_rna[-dup_id, ]

#try this
all_rna=all_rna[rowSums(all_rna)>50,colSums(all_rna)>400]

dim(all_rna)

In [15]:
# read file containing HTO identity info (singlet, doublet, etc)
hto <- read.csv("CZI.PMBC.HTO.pooled.sample.classifications.GMM.csv",
                header = T, check.names = F, stringsAsFactors = F, row.names = 2)
#change column names of HTO matrix, the prefixes of the columns in the HTO matrix need to match those of the RNA matrix
rownames(hto) <- gsub(x = rownames(hto), pattern = "HTO", replacement = "Sample")
# should have ~300k plus cell barcodes
length(which(colnames(all_rna) %in% rownames(hto)))

# read in adt/protein matrix data, this will also be used to cluster data
adt_counts <- readRDS("CZI.PMBC.ADT.pooled.combined.raw.matrix.Rds")
colnames(adt_counts) <- gsub(x = colnames(adt_counts), pattern = "ADT", replacement = "Sample")

# some rows in the ADT matrix contain summary info and not levels of specific proteins, so remove them
adt_id <- which(rownames(adt_counts) %in% c("bad_struct", "no_match", "total_reads"))
adt_counts <- adt_counts[-adt_id,]
# remove control ids too (these aren't useful)
adt_cont <- which(grepl(x = rownames(adt_counts), pattern = "control"))
adt_counts <- adt_counts[-adt_cont, ]
rownames(adt_counts) <- paste("CITE", rownames(adt_counts), sep = "-")

# change underscores in ADT row names to hyphens
rownames(adt_counts) <- gsub(x = rownames(adt_counts), pattern = "_", replacement = "-")

# analysis for all cells
hto_sel <- hto

#TRY: taking out the empty droplets from HTO None s
no_none_barcodes=rownames(hto[as.character(hto$Sample) != "None",]) #cells who doesn't have none barcode
cells_w_nb=Reduce(intersect,list(colnames(all_rna),no_none_barcodes)) #cells with no none barcodes overlapping with rna
no_none_ids=which(colnames(all_rna) %in% cells_w_nb) #where do these cells located in rna matrix
all_rna=all_rna[,no_none_ids] #at the end makes only 700 difference lol

# find intersection of all cell barcodes across HTO, RNA, and ADT matrices
int_cells <- Reduce(intersect, list(rownames(hto_sel), colnames(all_rna), colnames(adt_counts)))
rna_ids <- which(colnames(all_rna) %in% int_cells)
rna_sel <- all_rna[, rna_ids]
adt_sel <- adt_counts[, colnames(rna_sel)]
hto_sel <- hto[colnames(rna_sel), ]

write.csv(rownames(adt_sel),"ftrs_list.csv")

# do some filtering of the gene matrix (genes with low counts aren't useful for clustering)
rna_sel <- rna_sel[Matrix::rowSums(rna_sel) > 5,]
#rna_sel[rowSums(rna_sel)>40,colSums(rna_sel)>400]

# rename metadata column names
colnames(hto_sel) <- c("ID", "Markers", "Treatment", "Run")

# combine rna-seq and adt into a single matrix
comb.counts <- Matrix::rbind2(rna_sel, as.matrix(adt_sel))

[1] 219119

In [16]:
cc1=comb.counts[,substr(colnames(comb.counts)[],1,2)=="1-"]
cc2=comb.counts[,substr(colnames(comb.counts)[],1,2)=="2-"]
cc3=comb.counts[,substr(colnames(comb.counts)[],1,2)=="3-"]
cc4=comb.counts[,substr(colnames(comb.counts)[],1,2)=="4-"]
cc5=comb.counts[,substr(colnames(comb.counts)[],1,2)=="5-"]
cc6=comb.counts[,substr(colnames(comb.counts)[],1,2)=="6-"]
cc7=comb.counts[,substr(colnames(comb.counts)[],1,2)=="7-"]
cc8=comb.counts[,substr(colnames(comb.counts)[],1,2)=="8-"]
cc9=comb.counts[,substr(colnames(comb.counts)[],1,2)=="9-"]
cc10=comb.counts[,substr(colnames(comb.counts)[],1,2)=="10"]

In [18]:
hto1=hto_sel[substr(rownames(hto_sel)[],1,2)=="1-",]
hto2=hto_sel[substr(rownames(hto_sel)[],1,2)=="2-",]
hto3=hto_sel[substr(rownames(hto_sel)[],1,2)=="3-",]
hto4=hto_sel[substr(rownames(hto_sel)[],1,2)=="4-",]
hto5=hto_sel[substr(rownames(hto_sel)[],1,2)=="5-",]
hto6=hto_sel[substr(rownames(hto_sel)[],1,2)=="6-",]
hto7=hto_sel[substr(rownames(hto_sel)[],1,2)=="7-",]
hto8=hto_sel[substr(rownames(hto_sel)[],1,2)=="8-",]
hto9=hto_sel[substr(rownames(hto_sel)[],1,2)=="9-",]
hto10=hto_sel[substr(rownames(hto_sel)[],1,2)=="10",]

In [51]:
saveRDS(CreateSeuratObject(counts=cc1,meta.data=hto1,assay="RNA"),"1_Seur_Input.Rds")
saveRDS(CreateSeuratObject(counts=cc2,meta.data=hto2,assay="RNA"),"2_Seur_Input.Rds")
saveRDS(CreateSeuratObject(counts=cc3,meta.data=hto3,assay="RNA"),"3_Seur_Input.Rds")
saveRDS(CreateSeuratObject(counts=cc4,meta.data=hto4,assay="RNA"),"4_Seur_Input.Rds")
saveRDS(CreateSeuratObject(counts=cc5,meta.data=hto5,assay="RNA"),"5_Seur_Input.Rds")
saveRDS(CreateSeuratObject(counts=cc6,meta.data=hto6,assay="RNA"),"6_Seur_Input.Rds")
saveRDS(CreateSeuratObject(counts=cc7,meta.data=hto7,assay="RNA"),"7_Seur_Input.Rds")
saveRDS(CreateSeuratObject(counts=cc8,meta.data=hto8,assay="RNA"),"8_Seur_Input.Rds")
saveRDS(CreateSeuratObject(counts=cc9,meta.data=hto9,assay="RNA"),"9_Seur_Input.Rds")
saveRDS(CreateSeuratObject(counts=cc10,meta.data=hto10,assay="RNA"),"10_Seur_Input.Rds")

Warning message:
“Feature names cannot have underscores ('_'), replacing with dashes ('-')”Warning message:
“Feature names cannot have underscores ('_'), replacing with dashes ('-')”Warning message:
“Feature names cannot have underscores ('_'), replacing with dashes ('-')”Warning message:
“Feature names cannot have underscores ('_'), replacing with dashes ('-')”Warning message:
“Feature names cannot have underscores ('_'), replacing with dashes ('-')”Warning message:
“Feature names cannot have underscores ('_'), replacing with dashes ('-')”Warning message:
“Feature names cannot have underscores ('_'), replacing with dashes ('-')”Warning message:
“Feature names cannot have underscores ('_'), replacing with dashes ('-')”Warning message:
“Feature names cannot have underscores ('_'), replacing with dashes ('-')”Warning message:
“Feature names cannot have underscores ('_'), replacing with dashes ('-')”

In [50]:
writeMM(cc1, paste(name, "1_Scr_Input.mtx", sep = "."))
writeMM(cc2, paste(name, "2_Scr_Input.mtx", sep = "."))
writeMM(cc3, paste(name, "3_Scr_Input.mtx", sep = "."))
writeMM(cc4, paste(name, "4_Scr_Input.mtx", sep = "."))
writeMM(cc5, paste(name, "5_Scr_Input.mtx", sep = "."))
writeMM(cc6, paste(name, "6_Scr_Input.mtx", sep = "."))
writeMM(cc7, paste(name, "7_Scr_Input.mtx", sep = "."))
writeMM(cc8, paste(name, "8_Scr_Input.mtx", sep = "."))
writeMM(cc9, paste(name, "9_Scr_Input.mtx", sep = "."))
writeMM(cc10, paste(name, "10_Scr_Input.mtx", sep = "."))

NULL

NULL

NULL

NULL

NULL

NULL

NULL

NULL

NULL

NULL

In [44]:
source("df_pipe.R")

In [19]:
#ftrs_lst=read.csv("ftrs_list.csv")[,2]

#pbmc= readRDS(name)
pbmc = CreateSeuratObject(counts=cc1,meta.data = hto1,assay="RNA")
pbmc <- NormalizeData(object = pbmc, normalization.method = "CLR")

Warning message:
"Feature names cannot have underscores ('_'), replacing with dashes ('-')"Normalizing across features


In [2]:
pbmc1=readRDS("/projects/ucar-lab/danaco/bncmrk-dblts/DoubletDecon/input/CZI.PBMC/CZI.PBMC.6.raw.seurat.Input.for.DoubDec.Rds")

In [5]:
pbmc1[["RNA"]]@data[1:2,1:2]

2 x 2 sparse Matrix of class "dgCMatrix"
         6-Sample_AAACCTGAGAATGTGT 6-Sample_AAACCTGAGACAGAGA
A1BG                             .                         .
A1BG-AS1                         .                         .

In [11]:
writeMM(pbmc1@assays$RNA@counts,"CZI.1.Donor.RNA.Count.Expression.File.txt")

NULL

In [19]:
pebemece1=readMM(file='CZI.1.Donor.RNA.Count.Expression.File.txt')


In [6]:
pbmc1[['RNA']]@data[1:5,1:5]

5 x 5 sparse Matrix of class "dgCMatrix"
         6-Sample_AAACCTGAGAATGTGT 6-Sample_AAACCTGAGACAGAGA
A1BG                             .                         .
A1BG-AS1                         .                         .
A2M                              .                         .
A2M-AS1                          .                         .
A2ML1                            .                         .
         6-Sample_AAACCTGAGAGGTTAT 6-Sample_AAACCTGAGAGTTGGC
A1BG                             .                         .
A1BG-AS1                         .                         .
A2M                              .                         .
A2M-AS1                          .                         .
A2ML1                            .                         .
         6-Sample_AAACCTGAGCACCGTC
A1BG                             .
A1BG-AS1                         .
A2M                              .
A2M-AS1                          .
A2ML1                            .

In [4]:
mrkrs1=FindAllMarkers(pbmc1)

Calculating cluster 6-Sample
Warning message:
"No DE genes identified"Warning message:
"The following tests were not performed: "Warning message:
"When testing 6-Sample versus all:
	Cell group 2 is empty - no cells with identity class "

In [15]:
mrkrs1[1:5,]

,p_val,avg_logFC,pct.1,pct.2,p_val_adj,cluster,gene
CITE-CD4-TGTTCCCGCTCAACT,0,0.4245411,0.999,0.941,0,0,CITE-CD4-TGTTCCCGCTCAACT
CITE-CD3-CTCATTGTAACTCCT,0,0.3990192,1.000,0.998,0,0,CITE-CD3-CTCATTGTAACTCCT
CITE-CD183-CXCR3-GCGATGGTAGATTAT,0,-0.3343299,1.000,1.000,0,0,CITE-CD183-CXCR3-GCGATGGTAGATTAT
CITE-CD196-CCR6-GATCCCTTTGTCACT,0,-0.3705907,1.000,1.000,0,0,CITE-CD196-CCR6-GATCCCTTTGTCACT
MT-CO3,0,-0.4289946,0.923,0.966,0,0,MT-CO3


In [31]:
write.table(top_n(n=50,x=mrkrs1,wt="avg_logFC"),"Top50Genes.txt",sep="\t")

In [15]:
write.table(pbmc1@active.ident,"Cluster.txt",sep="\t",col.names=NA)

In [33]:
write.table(GetAssayData(pbmc1),"counts.txt",sep="\t", col.names=NA)

In [35]:
devtools::install_github('EDePasquale/DoubletDecon')


from URL https://api.github.com/repos/EDePasquale/DoubletDecon/zipball/master
Installation failed: error in running command


In [38]:
devtools::install_github("r-lib/remotes")

from URL https://api.github.com/repos/r-lib/remotes/zipball/master
Installation failed: error in running command


In [37]:
write.table(GetAssayData(pbmc1),sep="\t",row.names=T,file="ekspiresin1.txt")

In [45]:
library(devtools)
options(unzip = "internal")
devtools::install_github('EDePasquale/DoubletDecon')

from URL https://api.github.com/repos/EDePasquale/DoubletDecon/zipball/master
Installing DoubletDecon
'/home/danaco/.conda/envs/scSplit/lib/R/bin/R' --no-site-file --no-environ  \
  --no-save --no-restore --quiet CMD INSTALL  \
  '/scratch/10297577.helix-master/RtmpkmCnhY/devtools5a1d32b0bc0a/EDePasquale-DoubletDecon-2549f49'  \
  --library='/home/danaco/R/x86_64-pc-linux-gnu-library/3.4' --install-tests 

Installation failed: Command failed (1)


In [1]:
pbmc1_rna=read.table("counts.txt")

In [2]:
colnames(pbmc1_rna)[1]

[1] "X1.Sample_AAACCTGAGAATGTGT"

In [14]:
pbmc1[['RNA']]@data[1:5,1:5]

ERROR: Error in eval(expr, envir, enclos): object 'pbmc1' not found


In [2]:
  expression=read.table("ekspiresin1.txt", sep="\t",header=T, row.names=1)


In [4]:
mehmet=colnames(expression)[1]

In [8]:
colnames(expression)[1]

[1] "X1.Sample_AAACCTGAGAATGTGT"

In [10]:
pbmc=readRDS("/projects/ucar-lab/danaco/bncmrk-dblts/DoubletDecon/input/CZI.PBMC/CZI.PBMC.6.raw.seurat.Input.for.DoubDec.Rds")

In [3]:
pebemece.data=GetAssayData(pebemece)

In [11]:
pbmc <- NormalizeData(object = pbmc, normalization.method = "CLR")
pbmc <- FindVariableFeatures(object = pbmc, selection.method = 'vst', nfeatures = 500)
pbmc <- ScaleData(object = pbmc, features = rownames(x = pbmc))
print("Running PCA")
pbmc= RunPCA(pbmc,features = VariableFeatures(object = pbmc), verbose = FALSE)
print("Running UMAP")
pbmc <- RunUMAP(object = pbmc, dims = 1:10)
pbmc <- FindNeighbors(object = pbmc, dims = 1:10)
pbmc <- FindClusters(object = pbmc, resolution = 1.2)

Normalizing across features
Centering and scaling data matrix


[1] "Running PCA"
[1] "Running UMAP"


Computing nearest neighbor graph
Computing SNN


Modularity Optimizer version 1.3.0 by Ludo Waltman and Nees Jan van Eck

Number of nodes: 32138
Number of edges: 935284

Running Louvain algorithm...
Maximum modularity in 10 random starts: 0.8414
Number of communities: 29
Elapsed time: 53 seconds


In [116]:
cm=mrkrs

In [117]:
#  TOP_MARKERS 

#  cluster marker info
#cm=FindAllMarkers(so)
cm$cluster=as.character(cm$cluster)

#  cut to topNum for each cluster
TopNum=50

nclusters=length(unique(cm$cluster))
MperC=as.vector(table(cm$cluster))   #  Markers per Cluster
MperC=ifelse(MperC >TopNum,TopNum,MperC)

#  Accumulate the genes (features) 
genesFile=NULL
for (i in 1:(nclusters-1))
{
  a=cm[1:MperC[i],] # TopNum genes for cluster i
  c=a$cluster     #  capture cluster identifier
  genesFile=rbind.data.frame(genesFile,a,stringsAsFactors = F)
  #  remove data for cluster just processed
  cm=cm[! (cm$cluster %in% c),]
}
#genesFile$cluster=as.numeric(genesFile$cluster)
genesFile$cluster=as.character(genesFile$cluster)

#rm(cm,a,c)

 table(genesFile$cluster)  #  a check
#  note that genesFile$cluster numbering starts at 1 !






 0  1 10 11 12 13 14 15 16 17 18 19  2 20 21 22 23 24 25 26 27 28  3  4  5  6 
50 50 50 50 50 50 50 50 50 50 50 50 50 37 50 50 50 50 50 23 50 50 50 50 50 23 
 7  8  9 
27 50 50 

In [83]:
write.csv(genesFile,"tryGenesFile.csv")

In [113]:
write.table(genesFile,paste("CZI.PBMC.6.try","Top50Genes.txt",sep="."),sep="\t")#,col.names = NA) 


In [98]:
write.table(mrkrs,"CZI.PBMC.6.try.allmarkers.txt",sep="\t")#,col.names = NA) 


In [102]:
genesFile$""=as.character(1:(dim(genesFile)[1]))

In [106]:
bebe=top_n(n=50,x=mrkrs,wt="cluster")

In [107]:
write.table(bebe,"bebe.txt",sep="\t")

In [3]:
source("/projects/ucar-lab/danaco/bncmrk-dblts/R/Seurat_Pre_Process2.R")

In [ ]:
source("/projects/ucar-lab/danaco/bncmrk-dblts/R/Top50Markers.R")

name="CZI.PBMC.6.try"

write.table(GetAssayData(pbmc),paste(name,"counts.txt",sep="."),sep="\t", col.names=NA)
    
write.table(pbmc@active.ident,paste(name,"Cluster.txt",sep="."),sep="\t",col.names=NA)    

#cm=FindAllMarkers(pbmc) #cluster marker info
    
write.csv(mrkrs,paste(name,"AllMarkers.csv",sep="."))

topmrkrs=Top50Markers(mrkrs)

write.table(topmrkrs,paste(name,"Top50Genes.txt",sep="."),sep="\t") 

In [4]:
expressionFile="CZI.PBMC.6.try.counts.txt"
genesFile="CZI.PBMC.6.try.Top50Genes.txt"
clustersFile="CZI.PBMC.6.try.Cluster.txt"

newFiles=Seurat_Pre_Process2(expressionFile, genesFile, clustersFile)


In [1]:
library(Biobase)

Loading required package: BiocGenerics
Loading required package: parallel

Attaching package: 'BiocGenerics'

The following objects are masked from 'package:parallel':

    clusterApply, clusterApplyLB, clusterCall, clusterEvalQ,
    clusterExport, clusterMap, parApply, parCapply, parLapply,
    parLapplyLB, parRapply, parSapply, parSapplyLB

The following objects are masked from 'package:stats':

    IQR, mad, sd, var, xtabs

The following objects are masked from 'package:base':

    Filter, Find, Map, Position, Reduce, anyDuplicated, append,
    as.data.frame, cbind, colMeans, colSums, colnames, do.call,
    duplicated, eval, evalq, get, grep, grepl, intersect, is.unsorted,
    lapply, lengths, mapply, match, mget, order, paste, pmax, pmax.int,
    pmin, pmin.int, rank, rbind, rowMeans, rowSums, rownames, sapply,
    setdiff, sort, table, tapply, union, unique, unsplit, which,
    which.max, which.min

Welcome to Bioconductor

    Vignettes contain introductory material; view with
  

In [12]:
genesFileR=read.table("CZI.PBMC.6.try.Top50Genes.txt",row.names = 1,sep = "\t",header=TRUE)


In [16]:
dim(genesFileR)

[1] 1360    7

In [17]:
for (ii in c(6,8)){
    print(ii)
}

[1] 6
[1] 8
